<a href="https://colab.research.google.com/github/AtifQureshi110/webScraping/blob/main/Nike_men_shows_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Link**

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
# Initialize a list to store used url_ids
used_url_ids = []

def get_request(url_id, parser_id, soup_variable):
    global used_url_ids

    # Check if the url_id has been used before
    if url_id in used_url_ids:
        return "This url_id has already been used."

    res = requests.get(url_id)
    if res.status_code == 200:
        globals()[soup_variable] = bs(res.text, parser_id)
        used_url_ids.append(url_id)  # Add the url_id to the used list
        return url_id, res, type(res)
    else:
        return "Request failed."


# **Extraction functions**

In [ ]:
item_info=[]
def get_basic_info(soup_id, tag_id, class_id):
  show_info = soup_id.find_all(tag_id,class_=class_id)
  for detail in show_info:
    h1_element = detail.find("h1",class_="headline-2")
    if h1_element:
      item_info.append(h1_element.text)
    h2_element = detail.find("h2",class_="headline-5 pb1-sm d-sm-ib")
    if h2_element:
      item_info.append(h2_element.text)
    div_element = detail.find("div",class_="product-price css-11s12ax is--current-price css-tpaepq")
    if div_element:
      div_ele=div_element.text
      item_info.append(div_ele)
# get_basic_info(soup1,"div","pr2-sm css-1ou6bb2")
# item_info

In [ ]:
description_ls=[]
def get_description(soup_id, tag_id, class_id):
  info = soup_id.find_all(tag_id, class_=class_id)
  for t in  info:
    Para = t.find("p")
    if Para:
      description_ls.append(Para.text)
    color = t.find("li",class_="description-preview__color-description ncss-li")
    if color:
      description_ls.append(color.text)
    style = t.find("li",class_="description-preview__style-color ncss-li")
    if style:
      description_ls.append(style.text)
  #print("original list",description_ls)
  color = description_ls[1].replace("Shown: ", "")
  description_ls[1] = color
  style = description_ls[2].replace("Style: ", "")
  description_ls[2] = style
# get_description(soup1,"div","pt6-sm prl6-sm prl0-lg")
# description_ls

In [ ]:
detail_ls=[]
def get_detail(soup_id, tag_id, class_id):
  reviews = soup_id.find_all(tag_id, class_id)
  for review in reviews:
    policy = review.find("span", class_="exp-pdp-promo-message-text")
    order_link = review.find("a")
    if policy and order_link:
      detail_ls.append(policy.text)
      detail_ls.append(order_link.attrs['href'])      
# get_detail(soup1,"div", "exp-pdp-promo-message-text-link-wrapper")
# detail_ls

In [ ]:
rating_ls =[]
def get_rating(soup_id, tag_id, class_id):
  reviews = soup_id.find_all(tag_id, class_id)
  for review in reviews:
    rate = review.find("p", class_="d-sm-ib pl4-sm")
    if rate:
      rating_ls.append(rate.text)
    else:
      rating_ls.append("0 Stars")
# get_rating(soup1,"div", "reviews-component mb5-sm")
# rating_ls

In [ ]:
img_ls=[]
def get_img(soup_id, tag_id, class_id):
  imgs = soup_id.find_all(tag_id,class_id)
  for img in imgs:
    pic = img.find("img")
    if pic:
      img_ls.append(pic.attrs['src'])
# get_img(soup1, "div", "css-du206p")
# img_ls

In [ ]:
reviwe_handings=[]
reviwe_id_date=[]
reviwe_comment=[]
authors_ls = []
dates_ls = []
reviews_ls = []

def get_reviews_details(soup_id, tag_id, class_id):
  reviews = soup_id.find_all(tag_id, class_id)

  for review in reviews:
      h4s = review.find_all("h4")
      if h4s:
        for h4 in h4s:
          reviwe_handings.append(h4.text)
      else:
        reviwe_handings.append(None)

      paras = review.find_all("p", class_="pl4-sm d-sm-ib text-color-secondary responsive-body-3-2")
      if paras:
        for para in paras:
          author_date = para.text.split(' - ')
          if len(author_date) == 2:
            author = author_date[0]
            date = author_date[1]
            authors_ls.append(author)
            dates_ls.append(date)
          else:
            authors_ls.append(None)
            dates_ls.append(None)
      else:
        authors_ls.append(None)
        dates_ls.append(None)    

      comments = review.find_all("div")
      if comments:
        for comment in comments:
          if comment.text.strip() != '':
            reviews_ls.append(comment.text.strip())
      else:
        reviews_ls.append(None)    

In [ ]:
def get_extraction_link(soup):
  get_basic_info(soup,"div","pr2-sm css-1ou6bb2")    
  get_img(soup, "div", "css-du206p")   
  get_detail(soup,"div", "exp-pdp-promo-message-text-link-wrapper") 
  get_description(soup,"div","pt6-sm prl6-sm prl0-lg")
  get_rating(soup,"div", "reviews-component mb5-sm")
  print(f"{len(item_info)}{item_info}\n{len(img_ls)}{(img_ls)}\n{len(detail_ls)}{(detail_ls)}\n{len(description_ls)}{(description_ls)}\n{len(rating_ls)}{(rating_ls)}")
  get_reviews_details(soup, "li","review mb10-sm")
  print(f"{len(reviwe_handings)}{reviwe_handings}\n{len(dates_ls)}{dates_ls}\n{len(authors_ls)}{authors_ls}\n{len(reviews_ls)}{reviews_ls}")

In [ ]:
def create_dataframe(df_name, item_info, img_ls, description_ls, detail_ls, rating_ls):
    if len(rating_ls) == 0:
        rating_value = "0 Stars"
    else:
        rating_value = rating_ls[0]

    df = pd.DataFrame({"product_name": [item_info[0]], "category": [item_info[1]], "price": [item_info[2]],
                       "image": [img_ls[0]],
                       "description": [description_ls[0]], "colors": [description_ls[1]], "style": [description_ls[2]],
                       "policy": [detail_ls[0]],"order_link": [detail_ls[1]],
                       "rating": [rating_value]    })

    globals()[df_name] = df
# create_dataframe("link_1_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
# display(link_1_df)


In [ ]:
def create_review_dataframe(df_name, reviwe_handings, authors_ls, dates_ls, reviews_ls):
    df = pd.DataFrame({        "product_reviwe_title": reviwe_handings, "author": authors_ls, "date": dates_ls, "reviwe": reviews_ls[0:3]    })    
    globals()[df_name] = df

In [ ]:
def get_clear_list(list_ids):
  cleared = False  # Flag to track if any list was cleared  
  for list_id in list_ids:
    if len(list_id) > 0:
      list_id.clear()
      cleared = True  
  if cleared:
    print("All lists are cleared for new data arrival")
  else:
    print("Lists are already empty")
#get_clear_list([item_info, img_ls, description_ls, rating_ls])

# **extracted Information Nike Men shows**

## **link1data**

In [ ]:
# Create an empty list to store DataFrame names
dataframe_names = []
review_dataframe_names = []

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

Lists are already empty


In [ ]:
get_request("https://www.nike.com/t/air-jordan-low-se-mens-shoes-Ljd2gw/DX4334-008", 'html.parser', 'soup1')

('https://www.nike.com/t/air-jordan-low-se-mens-shoes-Ljd2gw/DX4334-008',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup1)

3['Air Jordan Low SE', "Men's Shoes", '$120']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/c50496c8-927b-414f-9c0d-9b3b62b987c1/air-jordan-low-se-mens-shoes-Ljd2gw.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Get into some summery fun in your new fave AJ1s. Made with a combination of suede and canvas, this pair gives you the comfort you know and love with a seasonal update.', 'Black/Magic Ember/White/Sundial', 'DX4334-008']
1['0 Stars']
0[]
0[]
0[]
0[]


In [ ]:
create_dataframe("link_1_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_1_df)
# Add the DataFrame name to the list
dataframe_names.append(link_1_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan Low SE,Men's Shoes,$120,https://static.nike.com/a/images/t_PDP_LOADING...,Get into some summery fun in your new fave AJ1...,Black/Magic Ember/White/Sundial,DX4334-008,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,0 Stars


In [ ]:
create_review_dataframe("review_1_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_1_df)
review_dataframe_names.append(review_1_df)

,product_reviwe_title,author,date,reviwe


## **link2data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-vapormax-plus-mens-shoes-nC0dzF/924453-004", 'html.parser', 'soup2')

('https://www.nike.com/t/air-vapormax-plus-mens-shoes-nC0dzF/924453-004',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup2)

3['Nike Air VaporMax Plus', "Men's Shoes", '$210']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/fliho8ygw5nogbmtcbft/air-vapormax-plus-mens-shoes-nC0dzF.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['The Nike Air VaporMax Plus looks to the past and propels you into the future. Nodding to the 1998 Air Max Plus with its floating cage, padded upper and heel logo, it adds revolutionary VaporMax Air technology to ramp up the comfort and create a modern look.', 'Black/Black/Dark Grey', '924453-004']
1['4.9 Stars']
3['Nike Air Vapor max plus', 'He loves it', 'Great features']
3['May 21, 2023', 'May 03, 2023', 'May 03, 2023']
3['Krista339132879', 'Ok1kidkog', 'Cbeech mallen']
5['I love these shoes and this is my 2nd purchase. They are super comfortable and so stylish, also give great support to my ankle and arch support. The only nega

In [ ]:
create_dataframe("link_2_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_2_df)
dataframe_names.append(link_2_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air VaporMax Plus,Men's Shoes,$210,https://static.nike.com/a/images/t_PDP_LOADING...,The Nike Air VaporMax Plus looks to the past a...,Black/Black/Dark Grey,924453-004,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.9 Stars


In [ ]:
create_review_dataframe("review_2_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_2_df)
review_dataframe_names.append(review_2_df)

,product_reviwe_title,author,date,reviwe
0,Nike Air Vapor max plus,Krista339132879,"May 21, 2023",I love these shoes and this is my 2nd purchase...
1,He loves it,Ok1kidkog,"May 03, 2023",[This review was collected as part of a promot...
2,Great features,Cbeech mallen,"May 03, 2023","Product received for free, or reviewed as part..."


## **link3data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-max-270-mens-shoes-KkLcGR/AH8050-100", 'html.parser', 'soup3')

('https://www.nike.com/t/air-max-270-mens-shoes-KkLcGR/AH8050-100',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup3)

3['Nike Air Max 270', "Men's Shoes", '$160']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/awjogtdnqxniqqk0wpgf/air-max-270-mens-shoes-KkLcGR.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Nike's first lifestyle Air Max brings you style, comfort and big attitude in the Nike Air Max 270. The design draws inspiration from Air Max icons, showcasing Nike's greatest innovation with its large window and fresh array of colors.", 'White/White/Black', 'AH8050-100']
1['4.7 Stars']
3['Awesome', 'Comfortable', 'Comfortable Fit']
3['May 11, 2023', 'May 10, 2023', 'May 09, 2023']
3['Steve420', 'Thebosslady', 'CSB95']
4['[This review was collected as part of a promotion.] Comfortable, rugged, versatile. I own 4 pairs, love the styles, the fit.', 'Product received for free, or reviewed as part of a sweepstakes/giveaway.\n#teamnike', 'I love ho

In [ ]:
create_dataframe("link_3_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_3_df)
dataframe_names.append(link_3_df)


,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Max 270,Men's Shoes,$160,https://static.nike.com/a/images/t_PDP_LOADING...,Nike's first lifestyle Air Max brings you styl...,White/White/Black,AH8050-100,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars


In [ ]:
create_review_dataframe("review_3_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_3_df)
review_dataframe_names.append(review_3_df)

,product_reviwe_title,author,date,reviwe
0,Awesome,Steve420,"May 11, 2023",[This review was collected as part of a promot...
1,Comfortable,Thebosslady,"May 10, 2023","Product received for free, or reviewed as part..."
2,Comfortable Fit,CSB95,"May 09, 2023","I love how these shoes look on my feet, they g..."


## **link4data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-jordan-5-retro-se-craft-mens-shoes-MGhD1k/FD9222-180", 'html.parser', 'soup4')

('https://www.nike.com/t/air-jordan-5-retro-se-craft-mens-shoes-MGhD1k/FD9222-180',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup4)

3['Air Jordan 5 Retro SE Craft', "Men's Shoes", '$210']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/9057c22b-8f60-4fa5-9418-73a358c47093/air-jordan-5-retro-se-craft-mens-shoes-MGhD1k.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Bring some pop to your look with the AJ5, the shoe that's recognizably cool. All the original elements are here: lace toggles, premium leather and that unmissable midsole. Updated with new colors and textures, here's a shoe that keeps every look fresh.", 'Light Orewood Brown/Flat Pewter/Sail/Safety Orange', 'FD9222-180']
1['5 Stars']
2['Fire 🔥', '']
2['May 23, 2023', 'May 23, 2023']
2['Trubz118', 'javierr906855513']
2['Comfy and stylish, we need more colors for this shoe 👍🏾👍🏾', 'I think we sleepin on these. Soft materials make these super comfortable. Textures and colors look clean once you got them 

In [ ]:
create_dataframe("link_4_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_4_df)
dataframe_names.append(link_4_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan 5 Retro SE Craft,Men's Shoes,$210,https://static.nike.com/a/images/t_PDP_LOADING...,"Bring some pop to your look with the AJ5, the ...",Light Orewood Brown/Flat Pewter/Sail/Safety Or...,FD9222-180,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,5 Stars


In [ ]:
create_review_dataframe("review_4_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_4_df)
review_dataframe_names.append(review_4_df)

,product_reviwe_title,author,date,reviwe
0,Fire 🔥,Trubz118,"May 23, 2023","Comfy and stylish, we need more colors for thi..."
1,,javierr906855513,"May 23, 2023",I think we sleepin on these. Soft materials ma...


## **link5data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/zoom-freak-4-basketball-shoes-dsLlqf/DJ6149-500", 'html.parser', 'soup5')

('https://www.nike.com/t/zoom-freak-4-basketball-shoes-dsLlqf/DJ6149-500',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup5)

3['Zoom Freak 4', 'Basketball Shoes', '$130']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/9989cd60-470b-4c37-b61c-d94a019819ce/zoom-freak-4-basketball-shoes-dsLlqf.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Giannis is an incessant storm of stamina and skill that keeps coming at opponents for 4 quarters or more. The forward-thinking design of his latest signature shoe helps propel you down the court in a lightweight fit that moves with you. It can handle quick changes in direction on both sides of the floor, giving you side-to-side stability and multidirectional traction as you Euro step to the hoop. This special colorway is inspired by the Greek Freak’s towering power and his ability to bring his inner fire to the floor night in and night out.', 'Indigo Haze/Blue Tint/Baltic Blue/Space Purple', 'DJ6149-500']
1['4.7 Stars'

In [ ]:
create_dataframe("link_5_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_5_df)
dataframe_names.append(link_5_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Zoom Freak 4,Basketball Shoes,$130,https://static.nike.com/a/images/t_PDP_LOADING...,Giannis is an incessant storm of stamina and s...,Indigo Haze/Blue Tint/Baltic Blue/Space Purple,DJ6149-500,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars


In [ ]:
create_review_dataframe("review_5_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_5_df)
review_dataframe_names.append(review_5_df)

,product_reviwe_title,author,date,reviwe
0,Amazing design,Edgardo586187082,"May 24, 2023",They're stilysh. You could totally wear them a...
1,good shoe,lilcp6,"May 21, 2023","great shoe, real comfy and light even tho gian..."
2,Cool design for women or men,Ritarose,"May 01, 2023",I purchased the shoes for myself in a size 8. ...


## **link6data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-max-pulse-mens-shoes-ShS3tL/DR0453-004", 'html.parser', 'soup6')

('https://www.nike.com/t/air-max-pulse-mens-shoes-ShS3tL/DR0453-004',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup6)

3['Nike Air Max Pulse', "Men's Shoes", '$150']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/fc1ac524-05f5-4273-8281-6031f63851a2/air-max-pulse-mens-shoes-ShS3tL.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['The Air Max Pulse pulls inspiration from the London music scene, bringing an underground touch to the iconic Air Max line. Its textile-wrapped midsole and vacuum-sealed accents keep ‘em looking fresh and clean, while colors inspired by the London music scene give your look the edge. Point-loaded Air cushioning—revamped from the incredibly plush Air Max 270—delivers better bounce, helping you push past your limits.', 'Cobblestone/Light Orewood Brown/Black/Reflect Silver', 'DR0453-004']
1['4.9 Stars']
3['Nike air max pulse', '🔥🔥🔥', "The New Daily's"]
3['Apr 22, 2023', 'Apr 06, 2023', 'Apr 04, 2023']
3['jacoba752203190', 'Kev

In [ ]:
create_dataframe("link_6_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_6_df)
dataframe_names.append(link_6_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Max Pulse,Men's Shoes,$150,https://static.nike.com/a/images/t_PDP_LOADING...,The Air Max Pulse pulls inspiration from the L...,Cobblestone/Light Orewood Brown/Black/Reflect ...,DR0453-004,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.9 Stars


In [ ]:
create_review_dataframe("review_6_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_6_df)
review_dataframe_names.append(review_6_df)

,product_reviwe_title,author,date,reviwe
0,Nike air max pulse,jacoba752203190,"Apr 22, 2023","Nice trainer great quality and comfortable, gr..."
1,🔥🔥🔥,KevinM682830475,"Apr 06, 2023",Super comfortable. 🔥🔥🔥
2,The New Daily's,Morgan C.,"Apr 04, 2023",I didn't take the shoes off for the rest of th...


## **link7data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-jordan-6-retro-shoes-QLGPHR/CT8529-600", 'html.parser', 'soup7')

('https://www.nike.com/t/air-jordan-6-retro-shoes-QLGPHR/CT8529-600',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup7)

3['Air Jordan 6 Retro', 'Shoes', '$200']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/ca4c3078-7982-4c62-b85f-15bbd34794a0/air-jordan-6-retro-shoes-QLGPHR.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["MJ's sixth signature shoe debuted during the 1990-1991 season as His Airness battled rivals in pursuit of an elusive first championship. Now, it's back in a variety of color schemes. Gear up for the 30th anniversary of the Air Jordan 6 with this timeless classic.", 'Varsity Red/Black', 'CT8529-600']
1['4.8 Stars']
3['Jordan…one of the best brands', 'Great color way', 'Red Oreos']
3['May 11, 2023', 'May 05, 2023', 'Apr 21, 2023']
3['Ace2112', 'A1miami', 'Sugarfoot']
4['[This review was collected as part of a promotion.] Love my Jordan 6 Retro’s in grey…been wanting a pair for the longest time. Look great and fit perfectly', 'Prod

In [ ]:
create_dataframe("link_7_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_7_df)
dataframe_names.append(link_7_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan 6 Retro,Shoes,$200,https://static.nike.com/a/images/t_PDP_LOADING...,MJ's sixth signature shoe debuted during the 1...,Varsity Red/Black,CT8529-600,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.8 Stars


In [ ]:
create_review_dataframe("review_7_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_7_df)
review_dataframe_names.append(review_7_df)

,product_reviwe_title,author,date,reviwe
0,Jordan…one of the best brands,Ace2112,"May 11, 2023",[This review was collected as part of a promot...
1,Great color way,A1miami,"May 05, 2023","Product received for free, or reviewed as part..."
2,Red Oreos,Sugarfoot,"Apr 21, 2023",Love these shoes goes with everything and very...


## **link8data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-max-1-86-og-g-nrg-mens-golf-shoes-3sqfzb/DX8437-106", 'html.parser', 'soup8')

('https://www.nike.com/t/air-max-1-86-og-g-nrg-mens-golf-shoes-3sqfzb/DX8437-106',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup8)

3["Nike Air Max 1 '86 OG G NRG", "Men's Golf Shoes", '$170']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/8390c022-8b80-487e-a7b9-49bfc352749c/air-max-1-86-og-g-nrg-mens-golf-shoes-3sqfzb.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['The holy grail of fresh hits the links in the Air Max 1 G. Featuring the 4-window design that lets you peak at the large Air unit underneath, this is on-course cool, confidence and comfort personified. It sports the vintage Air Max 1 details you love: Clean lines, no-sew overlays, unmistakable colors and melds them with our game-changing golf technology, so you can play all day, year-round in a timeless fit.', 'White/Hot Punch/Sanddrift/Gridiron', 'DX8437-106']
1['0 Stars']
0[]
0[]
0[]
0[]


In [ ]:
create_dataframe("link_8_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_8_df)
dataframe_names.append(link_8_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Max 1 '86 OG G NRG,Men's Golf Shoes,$170,https://static.nike.com/a/images/t_PDP_LOADING...,The holy grail of fresh hits the links in the ...,White/Hot Punch/Sanddrift/Gridiron,DX8437-106,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,0 Stars


In [ ]:
create_review_dataframe("review_8_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_8_df)
review_dataframe_names.append(review_8_df)

,product_reviwe_title,author,date,reviwe


## **link9data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/killshot-2-leather-mens-shoe-zrq1wk/432997-107", 'html.parser', 'soup9')

('https://www.nike.com/t/killshot-2-leather-mens-shoe-zrq1wk/432997-107',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup9)

3['Nike Killshot 2 Leather', "Men's Shoe", '$90']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/hvt3vtikntcpmkka2732/killshot-2-leather-mens-shoe-zrq1wk.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Inspired by the original low-profile tennis shoe, the Nike Killshot 2 updates the upper with a variety of textured leathers to create a fresh look. From soft suedes to smooth leathers with the perfect sheen, it's court-side attitude with a modern touch. To prove you're on top, the rubber gum sole adds the cherry on bottom.", 'Sail/Gum Yellow/Midnight Navy', '432997-107']
1['4.7 Stars']
3['Love the Killshot throwback look.  Build quality fell short.', 'I’ll wear these shoes for the rest of my life.', '']
3['Mar 27, 2023', 'Mar 02, 2023', 'Dec 09, 2022']
3['Norman', 'KerryH625017102', 'ChaseR239746242']
3['Bought a pair of Killshots 

In [ ]:
create_dataframe("link_9_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_9_df)
dataframe_names.append(link_9_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Killshot 2 Leather,Men's Shoe,$90,https://static.nike.com/a/images/t_PDP_LOADING...,Inspired by the original low-profile tennis sh...,Sail/Gum Yellow/Midnight Navy,432997-107,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars


In [ ]:
create_review_dataframe("review_9_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_9_df)
review_dataframe_names.append(review_9_df)

,product_reviwe_title,author,date,reviwe
0,Love the Killshot throwback look. Build quali...,Norman,"Mar 27, 2023",Bought a pair of Killshots 02/21 and wear thes...
1,I’ll wear these shoes for the rest of my life.,KerryH625017102,"Mar 02, 2023",Don’t ever stop making these shoes. I’ve bough...
2,,ChaseR239746242,"Dec 09, 2022",Please don't ever stop making these... every d...


## **link10data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/react-pegasus-trail-4-se-mens-trail-running-shoes-ccmnVP/FB7182-001", 'html.parser', 'soup10')

('https://www.nike.com/t/react-pegasus-trail-4-se-mens-trail-running-shoes-ccmnVP/FB7182-001',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup10)

3['Nike React Pegasus Trail 4 SE', "Men's Trail Running Shoes", '$140']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/bccb3393-9cb4-4897-985a-4a9a8ad377df/react-pegasus-trail-4-se-mens-trail-running-shoes-ccmnVP.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Running is your daily ritual, taking you from road to trail as you seek out new adventures and goals. Let the Nike React Pegasus Trail 4 take you there and back again with its supportive and springy feel. Underfoot traction keeps you going over rocky terrain, while still providing a smooth ride for the road. Midfoot support is controlled by the laces, letting you find the right fit and feel. Sharp, bright hues complement dark-room color neutrals, speaking to an inclusive world where digital and physical coexist in harmony. Your trusted workhorse with wings is ready for an o

In [ ]:
create_dataframe("link_10_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_10_df)
dataframe_names.append(link_10_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike React Pegasus Trail 4 SE,Men's Trail Running Shoes,$140,https://static.nike.com/a/images/t_PDP_LOADING...,"Running is your daily ritual, taking you from ...",Pure Platinum/Baltic Blue/Black/Green Abyss,FB7182-001,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.6 Stars


In [ ]:
create_review_dataframe("review_10_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_10_df)
review_dataframe_names.append(review_10_df)

,product_reviwe_title,author,date,reviwe
0,"A+ style, F for durability",Libby987654321,"May 23, 2023",Super cute. Love the color combinations. The d...
1,Perfect,GaryW426365816,"May 17, 2023","Actually really happy with these, normally use..."
2,So comfy,Kayryry,"May 10, 2023",[This review was collected as part of a promot...


## **link11data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/blazer-mid-77-vintage-mens-shoes-nw30B2/BQ6806-100", 'html.parser', 'soup11')

('https://www.nike.com/t/blazer-mid-77-vintage-mens-shoes-nw30B2/BQ6806-100',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup11)

3["Nike Blazer Mid '77 Vintage", "Men's Shoes", '$105']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/fb7eda3c-5ac8-4d05-a18f-1c2c5e82e36e/blazer-mid-77-vintage-mens-shoes-nw30B2.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['In the ‘70s, Nike was the new shoe on the block. So new in fact, we were still breaking into the basketball scene and testing prototypes on the feet of our local team. Of course, the design improved over the years, but the name stuck. The Nike Blazer Mid ’77 Vintage—classic since the beginning.', 'White/Black', 'BQ6806-100']
1['4.7 Stars']
3['80s Vibe', 'Great quality', 'Nice high tops']
3['Apr 27, 2023', 'Apr 23, 2023', 'Apr 18, 2023']
3['N0X9', 'Bash', 'Axeman83']
3['Way more comfortable than I was expecting. Also love the retro look and feel', 'I got my shoe really quick....best quality and fits perfect

In [ ]:
create_dataframe("link_11_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_11_df)
dataframe_names.append(link_11_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Blazer Mid '77 Vintage,Men's Shoes,$105,https://static.nike.com/a/images/t_PDP_LOADING...,"In the ‘70s, Nike was the new shoe on the bloc...",White/Black,BQ6806-100,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars


In [ ]:
create_review_dataframe("review_11_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_11_df)
review_dataframe_names.append(review_11_df)

,product_reviwe_title,author,date,reviwe
0,80s Vibe,N0X9,"Apr 27, 2023",Way more comfortable than I was expecting. Als...
1,Great quality,Bash,"Apr 23, 2023",I got my shoe really quick....best quality and...
2,Nice high tops,Axeman83,"Apr 18, 2023","Very happy with these, well made durable, easy..."


## **link12data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/dunk-high-retro-mens-shoes-dTVTCk/DV0829-300", 'html.parser', 'soup12')

('https://www.nike.com/t/dunk-high-retro-mens-shoes-dTVTCk/DV0829-300',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup12)

3['Nike Dunk High Retro', "Men's Shoes", '$125']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/aa3c8a05-e279-4686-88f0-08b60fd68e01/dunk-high-retro-mens-shoes-dTVTCk.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Recognizing the Dunk\'s roots as the top-ranking college team sneaker, the "Be True To Your School" pack looks to the original ad campaign for inspiration. Colors represent top-flight universities, while design details serve throwback hoops flair. And that crisp leather has the perfect amount of sheen to make \'em a hands-down win. So lace up and show off that varsity spirit with color combos that nod to your fave team. Ya game?', 'Stadium Green/Stadium Green/White/White', 'DV0829-300']
1['4.8 Stars']
3['Amazing experience', '', '']
3['May 02, 2023', 'Mar 29, 2023', 'Mar 11, 2023']
3['Dentory', 'Sneakypeek', 'MPete005'

In [ ]:
create_dataframe("link_12_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_12_df)
dataframe_names.append(link_12_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Dunk High Retro,Men's Shoes,$125,https://static.nike.com/a/images/t_PDP_LOADING...,Recognizing the Dunk's roots as the top-rankin...,Stadium Green/Stadium Green/White/White,DV0829-300,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.8 Stars


In [ ]:
create_review_dataframe("review_12_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_12_df)
review_dataframe_names.append(review_12_df)

,product_reviwe_title,author,date,reviwe
0,Amazing experience,Dentory,"May 02, 2023",Great shoe great people love the store I’m alw...
1,,Sneakypeek,"Mar 29, 2023",Best dunks I ever had. They are so crispy and ...
2,,MPete005,"Mar 11, 2023",No idea how these are still available in so ma...


## **link13data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/pegasus-40-mens-road-running-shoes-MCGW9h/FB7180-001", 'html.parser', 'soup13')

('https://www.nike.com/t/pegasus-40-mens-road-running-shoes-MCGW9h/FB7180-001',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup13)

3['Nike Pegasus 40 SE', "Men's Road Running Shoes", '$130']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/41a4652a-8447-4d1d-8a58-8797769eb423/pegasus-40-se-mens-road-running-shoes-MCGW9h.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['A springy ride for every run, the Peg’s familiar, just-for-you feel returns to help you accomplish your goals. This version has the same responsiveness and neutral support you love, but with improved comfort in those sensitive areas of your foot, like the arch and toes. Whether you’re logging long marathon miles, squeezing in a speed session before the sun goes down or hopping into a spontaneous group jaunt, it’s still the established road runner you can put your faith in, day after day, run after run. Sharp, bright hues complement dark-room color neutrals, speaking to an inclusive world where dig

In [ ]:
create_dataframe("link_13_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_13_df)
dataframe_names.append(link_13_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Pegasus 40 SE,Men's Road Running Shoes,$130,https://static.nike.com/a/images/t_PDP_LOADING...,"A springy ride for every run, the Peg’s famili...",Black/Green Abyss/Baltic Blue/Hyper Pink,FB7180-001,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars


In [ ]:
create_review_dataframe("review_13_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_13_df)
review_dataframe_names.append(review_13_df)

,product_reviwe_title,author,date,reviwe
0,Zoom,Zoom,"May 23, 2023",I gifted them to my sister. She is currently p...
1,Cute and comfy!,Maliha,"May 21, 2023","Product received for free, or reviewed as part..."
2,Pegasus 40,SR,"May 21, 2023",I really liked these sneakers! I think the des...


## **link14data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-jordan-1-retro-high-og-mens-shoes-JHpxkn/DZ5485-052", 'html.parser', 'soup14')

('https://www.nike.com/t/air-jordan-1-retro-high-og-mens-shoes-JHpxkn/DZ5485-052',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup14)

3['Air Jordan 1 Retro High OG', "Men's Shoes", '$180']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/e0a6c873-3bff-4cfe-9966-3bd8dedff1fd/air-jordan-1-retro-high-og-mens-shoes-JHpxkn.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["No such thing as a perfect marriage, but this one comes pretty close. The classic AJ1 silhouette gets blessed with the most iconic element of the AJ3—you know what it is—proving that two icons are made even better together. Black, White and Tech Grey create a modern neutral colorway that will set off any 'fit, and premium materials will keep you on your feet all day. Cheers to the happy couple.", 'Tech Grey/Black/White/Muslin', 'DZ5485-052']
1['4.9 Stars']
3['Sail laces for the win', 'Don’t sleep on these!', 'Nicely detailed']
3['May 18, 2023', 'May 12, 2023', 'May 07, 2023']
3['JohnH558317518', 'Taylo

In [ ]:
create_dataframe("link_14_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_14_df)
dataframe_names.append(link_14_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan 1 Retro High OG,Men's Shoes,$180,https://static.nike.com/a/images/t_PDP_LOADING...,"No such thing as a perfect marriage, but this ...",Tech Grey/Black/White/Muslin,DZ5485-052,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.9 Stars


In [ ]:
create_review_dataframe("review_14_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_14_df)
review_dataframe_names.append(review_14_df)

,product_reviwe_title,author,date,reviwe
0,Sail laces for the win,JohnH558317518,"May 18, 2023","The internet trashed these for being ""too flas..."
1,Don’t sleep on these!,TaylorT87323778,"May 12, 2023",Great quality!
2,Nicely detailed,Jojo2023,"May 07, 2023",Nice shoe love the print and off white sole. G...


## **link15data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-vapormax-2023-flyknit-mens-shoes-3q1qZg/DV1678-002", 'html.parser', 'soup15')

('https://www.nike.com/t/air-vapormax-2023-flyknit-mens-shoes-3q1qZg/DV1678-002',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup15)

3['Nike Air VaporMax 2023 Flyknit', "Men's Shoes", '$210']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/a108aee1-874a-486b-a5e8-7a8cc84fc373/air-vapormax-2023-flyknit-mens-shoes-3q1qZg.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Have you ever walked on Air? Step into the Air VaporMax 2023 to see how it's done. The innovative tech is revealed through the perforated sockliner (pull it out to see more). The stretchy Flyknit upper is made with at least 20% recycled content by weight.", 'Pure Platinum/Pure Platinum/Pure Platinum/White', 'DV1678-002']
1['5 Stars']
3['New to Vapourmax', 'Perfection', 'LOVE VAPORMAX']
3['May 17, 2023', 'May 04, 2023', 'May 01, 2023']
3['RogerH580662010', 'BenP286374118', 'SheShe09']
3['First pair I have purchased , true to size ,  comfortable , a great shoe at a reasonable price compared to other b

In [ ]:
create_dataframe("link_15_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_15_df)
dataframe_names.append(link_15_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air VaporMax 2023 Flyknit,Men's Shoes,$210,https://static.nike.com/a/images/t_PDP_LOADING...,Have you ever walked on Air? Step into the Air...,Pure Platinum/Pure Platinum/Pure Platinum/White,DV1678-002,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,5 Stars


In [ ]:
create_review_dataframe("review_15_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_15_df)
review_dataframe_names.append(review_15_df)

,product_reviwe_title,author,date,reviwe
0,New to Vapourmax,RogerH580662010,"May 17, 2023","First pair I have purchased , true to size , ..."
1,Perfection,BenP286374118,"May 04, 2023","Comfy, crazy fit… easily the best pair of Nike..."
2,LOVE VAPORMAX,SheShe09,"May 01, 2023",I am partial to Nike Vapormax! Love that I was...


## **link16data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-max-90-mens-shoes-6n3vKB/CN8490-002", 'html.parser', 'soup16')

('https://www.nike.com/t/air-max-90-mens-shoes-6n3vKB/CN8490-002',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup16)

3['Nike Air Max 90', "Men's Shoes", '$130']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/wzitsrb4oucx9jukxsmc/air-max-90-mens-shoes-6n3vKB.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Nothing as fly, nothing as comfortable, nothing as proven. The Nike Air Max 90 stays true to its OG running roots with the iconic Waffle sole, stitched overlays and classic TPU details. Classic colors celebrate your fresh look while Max Air cushioning adds comfort to the journey.', 'Iron Grey/Dark Smoke Grey/Black/White', 'CN8490-002']
1['4.7 Stars']
3['', 'Great shoes!', 'Good colorb']
3['May 03, 2023', 'May 01, 2023', 'Apr 25, 2023']
3['Budi41396905', 'nvoScorpio', 'B Boggs']
3["these shoes have an amazing color light to medium gray. comfortable. the front of the shoes squeak on my hardwood floors. hopefully it'll go away.", 'I bought these y

In [ ]:
create_dataframe("link_16_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_16_df)
dataframe_names.append(link_16_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Max 90,Men's Shoes,$130,https://static.nike.com/a/images/t_PDP_LOADING...,"Nothing as fly, nothing as comfortable, nothin...",Iron Grey/Dark Smoke Grey/Black/White,CN8490-002,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars


In [ ]:
create_review_dataframe("review_16_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_16_df)
review_dataframe_names.append(review_16_df)

,product_reviwe_title,author,date,reviwe
0,,Budi41396905,"May 03, 2023",these shoes have an amazing color light to med...
1,Great shoes!,nvoScorpio,"May 01, 2023",I bought these yesterday and I absolutely love...
2,Good colorb,B Boggs,"Apr 25, 2023",Bought it for my son and loved it. Purchased a...


## **link17data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-jordan-1-mid-se-mens-shoes-vw6Hxn/DX4332-800", 'html.parser', 'soup17')

('https://www.nike.com/t/air-jordan-1-mid-se-mens-shoes-vw6Hxn/DX4332-800',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup17)

3['Air Jordan 1 Mid SE', "Men's Shoes", '$135']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/22b7ce96-431b-432a-bd1a-ed32894a4430/air-jordan-1-mid-se-mens-shoes-vw6Hxn.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Get into some summery fun in your new fave AJ1s. Made with a combination of suede and canvas, this pair gives you the comfort you know and love with a seasonal update.', 'Magic Ember/White/Black/Sundial', 'DX4332-800']
1['0 Stars']
0[]
0[]
0[]
0[]


In [ ]:
create_dataframe("link_17_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_17_df)
dataframe_names.append(link_17_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan 1 Mid SE,Men's Shoes,$135,https://static.nike.com/a/images/t_PDP_LOADING...,Get into some summery fun in your new fave AJ1...,Magic Ember/White/Black/Sundial,DX4332-800,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,0 Stars


In [ ]:
create_review_dataframe("review_17_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_17_df)
review_dataframe_names.append(review_17_df)

,product_reviwe_title,author,date,reviwe


## **link18data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-force-1-07-mens-shoes-jBrhbr/CW2288-111", 'html.parser', 'soup18')

('https://www.nike.com/t/air-force-1-07-mens-shoes-jBrhbr/CW2288-111',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup18)

3["Nike Air Force 1 '07", "Men's Shoes", '$110']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/b7d9211c-26e7-431a-ac24-b0540fb3c00f/air-force-1-07-mens-shoes-jBrhbr.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['The radiance lives on in the Nike Air Force 1 ’07, the b-ball OG that puts a fresh spin on what you know best: durably stitched overlays, clean finishes and the perfect amount of flash to make you shine.', 'White/White', 'CW2288-111']
1['4.8 Stars']
3['Got 2 Keep A FRESH Pair In The Cut!!!', 'Classic of all Time', 'Air force 1']
3['May 05, 2023', 'May 05, 2023', 'May 04, 2023']
3['Tre4Zay', 'Linjiale413824465', 'VictoryS428041667']
4['[This review was collected as part of a promotion.] Bought these for my son & he loves them. Fit Is true to size and it’s always good to keep a new pair just in case you need them', 'Produ

In [ ]:
create_dataframe("link_18_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_18_df)
dataframe_names.append(link_18_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Force 1 '07,Men's Shoes,$110,https://static.nike.com/a/images/t_PDP_LOADING...,The radiance lives on in the Nike Air Force 1 ...,White/White,CW2288-111,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.8 Stars


In [ ]:
create_review_dataframe("review_18_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_18_df)
review_dataframe_names.append(review_18_df)

,product_reviwe_title,author,date,reviwe
0,Got 2 Keep A FRESH Pair In The Cut!!!,Tre4Zay,"May 05, 2023",[This review was collected as part of a promot...
1,Classic of all Time,Linjiale413824465,"May 05, 2023","Product received for free, or reviewed as part..."
2,Air force 1,VictoryS428041667,"May 04, 2023",Super comfortable and you just can't go wrong ...


## **link19data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/jordan-retro-6-g-nrg-mens-golf-shoes-0p95h4/FD0204-001", 'html.parser', 'soup19')

('https://www.nike.com/t/jordan-retro-6-g-nrg-mens-golf-shoes-0p95h4/FD0204-001',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup19)

3['Jordan Retro 6 G NRG', "Men's Golf Shoes", '$220']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/2fda8387-2b0b-440a-8fae-8bf428b91d08/jordan-retro-6-g-nrg-mens-golf-shoes-0p95h4.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Feel unbeatable from the tee box to the final putt in a design is pure early MJ: speed, class and laden with true early '90s touches. This model fuses the strut of 1st MJ’s championship with some of our best golf technology, helping you make a statement of confidence when it comes time to tame the course. This edgy, all-black design represents the grit mentality of the city that never sleeps, the no-quit persona that stretches from the streets to the skyscrapers.", 'Black/Metallic Silver/Chrome/Wolf Grey', 'FD0204-001']
1['5 Stars']
1['']
1['May 17, 2023']
1['thiessl968720828']
1['The golf division is cr

In [ ]:
create_dataframe("link_19_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_19_df)
dataframe_names.append(link_19_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Jordan Retro 6 G NRG,Men's Golf Shoes,$220,https://static.nike.com/a/images/t_PDP_LOADING...,Feel unbeatable from the tee box to the final ...,Black/Metallic Silver/Chrome/Wolf Grey,FD0204-001,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,5 Stars


In [ ]:
create_review_dataframe("review_19_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_19_df)
review_dataframe_names.append(review_19_df)

,product_reviwe_title,author,date,reviwe
0,,thiessl968720828,"May 17, 2023",The golf division is crushing it. I wish the l...


## **link20data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-max-97-mens-shoes-LJmK45/921826-101", 'html.parser', 'soup20')

('https://www.nike.com/t/air-max-97-mens-shoes-LJmK45/921826-101',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup20)

3['Nike Air Max 97', "Men's Shoes", '$175']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/a47b2ef9-8239-4e82-99fd-e6159c0df489/air-max-97-mens-shoes-LJmK45.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['Featuring the original ripple design inspired by Japanese bullet trains, the Nike Air Max 97 lets you push your style full-speed ahead. Taking the revolutionary full-length Nike Air unit that shook up the running world and adding fresh colors and crisp details, it lets you ride in first-class comfort.', 'White/Black/Wolf Grey', '921826-101']
1['4.8 Stars']
3['Let me tell ya', 'Comfortable', 'Air Max 97']
3['May 12, 2023', 'May 03, 2023', 'Apr 21, 2023']
3['Body by Marc', 'Tents____', 'A Phil']
5['[This review was collected as part of a promotion.] Bought the airmax 97s (black/white). Stylish, not too heavy. I use my air max for 

In [ ]:
create_dataframe("link_20_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_20_df)
dataframe_names.append(link_20_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Max 97,Men's Shoes,$175,https://static.nike.com/a/images/t_PDP_LOADING...,Featuring the original ripple design inspired ...,White/Black/Wolf Grey,921826-101,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.8 Stars


In [ ]:
create_review_dataframe("review_20_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_20_df)
review_dataframe_names.append(review_20_df)

,product_reviwe_title,author,date,reviwe
0,Let me tell ya,Body by Marc,"May 12, 2023",[This review was collected as part of a promot...
1,Comfortable,Tents____,"May 03, 2023","Product received for free, or reviewed as part..."
2,Air Max 97,A Phil,"Apr 21, 2023",[This review was collected as part of a promot...


## **link21data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-jordan-1-low-mens-shoes-0LXhbn/553558-174", 'html.parser', 'soup21')

('https://www.nike.com/t/air-jordan-1-low-mens-shoes-0LXhbn/553558-174',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup21)

3['Air Jordan 1 Low', "Men's Shoes", '$110']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/3f0a3ad8-dee0-4549-87d8-ee993bb11a99/air-jordan-1-low-mens-shoes-0LXhbn.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Inspired by the original that debuted in 1985, the Air Jordan 1 Low offers a clean, classic look that's familiar yet always fresh. With an iconic design that pairs perfectly with any 'fit, these kicks ensure you'll always be on point.", 'White/Aquatone/Celestial Gold', '553558-174']
1['4.9 Stars']
3['Need offerings in half sizes', 'It’s the green for me', 'Fire shoes !!']
3['May 23, 2023', 'May 10, 2023', 'May 07, 2023']
3['ljb36', 'Greenhouse', 'ant1816']
5["I love this shoe and have purchased multiple pairs in different colorways, both this version as well as for golf. I'm frustrated that it seems like the green/black a

In [ ]:
create_dataframe("link_21_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_21_df)
dataframe_names.append(link_21_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan 1 Low,Men's Shoes,$110,https://static.nike.com/a/images/t_PDP_LOADING...,"Inspired by the original that debuted in 1985,...",White/Aquatone/Celestial Gold,553558-174,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.9 Stars


In [ ]:
create_review_dataframe("review_21_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_21_df)
review_dataframe_names.append(review_21_df)

,product_reviwe_title,author,date,reviwe
0,Need offerings in half sizes,ljb36,"May 23, 2023",I love this shoe and have purchased multiple p...
1,It’s the green for me,Greenhouse,"May 10, 2023",[This review was collected as part of a promot...
2,Fire shoes !!,ant1816,"May 07, 2023","Product received for free, or reviewed as part..."


## **link22data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/air-max-plus-mens-shoes-x9G2xF/604133-139", 'html.parser', 'soup22')

('https://www.nike.com/t/air-max-plus-mens-shoes-x9G2xF/604133-139',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup22)

3['Nike Air Max Plus', "Men's Shoes", '$175']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/lx0owmisj943sr59emb8/air-max-plus-mens-shoes-x9G2xF.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Let your attitude have the edge in your Nike Air Max Plus, a Tuned Air experience that offers premium stability and unbelievable cushioning. Featuring the OG's wavy design lines, TPU accents and airy mesh on the upper, it celebrates defiant style.", 'White/Black/Cool Grey/White', '604133-139']
1['4.2 Stars']
3['Love the look, but way too narrow.', '', 'Ok shoe']
3['Feb 04, 2023', 'Nov 21, 2022', 'Nov 20, 2022']
3['SonyF90609204', 'Abdulrahman411047519', 'TMac79']
4["I had been wanting these for so long, so I'm disappointed. These are the most narrow shoes I've ever owned. I usually go up 1/2 size in Air Max's. Went up a full size with these

In [ ]:
create_dataframe("link_22_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_22_df)
dataframe_names.append(link_22_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Nike Air Max Plus,Men's Shoes,$175,https://static.nike.com/a/images/t_PDP_LOADING...,Let your attitude have the edge in your Nike A...,White/Black/Cool Grey/White,604133-139,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.2 Stars


In [ ]:
create_review_dataframe("review_22_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_22_df)
review_dataframe_names.append(review_22_df)

,product_reviwe_title,author,date,reviwe
0,"Love the look, but way too narrow.",SonyF90609204,"Feb 04, 2023","I had been wanting these for so long, so I'm d..."
1,,Abdulrahman411047519,"Nov 21, 2022",Comfy shoes
2,Ok shoe,TMac79,"Nov 20, 2022",[This review was collected as part of a promot...


## **link23data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/lebron-xx-four-horsemen-basketball-shoes-ct1qVm/DV9090-200", 'html.parser', 'soup23')

#https://www.nike.com/t/dunk-mid-mens-shoes-RHtFM2/DV0830-102

('https://www.nike.com/t/lebron-xx-four-horsemen-basketball-shoes-ct1qVm/DV9090-200',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup23)

3['LeBron XX "Four Horsemen"', 'Basketball Shoes', '$210']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/57e1545d-2191-477a-b0cb-842590d8ed1a/lebron-xx-four-horsemen-basketball-shoes-ct1qVm.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3["Two decades into a career that’s exceeded every lofty expectation, LeBron James has refused to settle for anything less than greatness, even when he was the one setting the standard for generations to come. The LeBron 20 is lighter, low to the ground and turbo-like. It’s unlike anything LeBron has donned before—supportive, yet low-cut, feathery-fast and created to stay ahead of today’s frenzied style of play. Cracked leather material upper and leather tongues highlight a design rooted in luxuriance whether you're on or off the hardwood. This version nods to the bond forged by LeBron and his clos

In [ ]:
create_dataframe("link_23_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_23_df)
dataframe_names.append(link_23_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,"LeBron XX ""Four Horsemen""",Basketball Shoes,$210,https://static.nike.com/a/images/t_PDP_LOADING...,Two decades into a career that’s exceeded ever...,Sesame/Sail/Midnight Navy/Light Crimson,DV9090-200,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,0 Stars


In [ ]:
create_review_dataframe("review_23_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_23_df)
review_dataframe_names.append(review_23_df)

,product_reviwe_title,author,date,reviwe


## **link24data**

In [ ]:
get_clear_list([item_info, img_ls, description_ls, detail_ls, rating_ls, reviwe_handings, dates_ls, authors_ls, reviews_ls])

All lists are cleared for new data arrival


In [ ]:
get_request("https://www.nike.com/t/tatum-1-barbershop-basketball-shoes-98vPwk/DX5571-180", 'html.parser', 'soup24')

#https://www.nike.com/t/dunk-mid-mens-shoes-RHtFM2/DV0830-102

('https://www.nike.com/t/tatum-1-barbershop-basketball-shoes-98vPwk/DX5571-180',
 <Response [200]>,
 requests.models.Response)

In [ ]:
get_extraction_link(soup24)

3['Tatum 1 "Barbershop"', 'Basketball Shoes', '$120']
1['https://static.nike.com/a/images/t_PDP_LOADING_v1/f_auto,q_auto:eco/47396608-fdf5-4175-86c3-35830bb4756f/tatum-1-barbershop-basketball-shoes-98vPwk.png']
2['Free standard shipping on orders $50+ and free 60-day returns for Nike Members. Learn more. Return policy exclusions apply.', 'https://www.nike.com/help/a/free-shipping']
3['It’s that feeling of getting a shape-up or a whole new \'do—that exciting, first-rate freshness. Jayson knows it well, and says it best: "When I get a cut, I think I\'m top 5." Channel that confidence on the court with a Tatum 1 that looks as fly as your fade. Neutrals keep it easy to wear while bright red and blue accents pay homage to the classic barbershop pole. The tongue pull tab shouts out Jay\'s son, Deuce. And the 5 stars on the heel? That\'s your cue to feel as good as you look.', 'Coconut Milk/Black/Laser Blue/Metallic Gold', 'DX5571-180']
1['5 Stars']
1['Favorite shoe']
1['May 10, 2023']
1['Pet

In [ ]:
create_dataframe("link_24_df", item_info, img_ls, description_ls, detail_ls, rating_ls)
display(link_24_df)
dataframe_names.append(link_24_df)

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,"Tatum 1 ""Barbershop""",Basketball Shoes,$120,https://static.nike.com/a/images/t_PDP_LOADING...,It’s that feeling of getting a shape-up or a w...,Coconut Milk/Black/Laser Blue/Metallic Gold,DX5571-180,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,5 Stars


In [ ]:
create_review_dataframe("review_24_df", reviwe_handings, authors_ls, dates_ls, reviews_ls)
display(review_24_df)
review_dataframe_names.append(review_24_df)

,product_reviwe_title,author,date,reviwe
0,Favorite shoe,PeteI41780660,"May 10, 2023",The Tatum 1 is my new favorite shoe. I'm a coa...


## **ImportantInformationOfNikeWebOfMenShowes**

In [ ]:
nike_men_shows_df = pd.concat(dataframe_names, ignore_index=True)
nike_men_shows_df
# dataframe_names

,product_name,category,price,image,description,colors,style,policy,order_link,rating
0,Air Jordan Low SE,Men's Shoes,$120,https://static.nike.com/a/images/t_PDP_LOADING...,Get into some summery fun in your new fave AJ1...,Black/Magic Ember/White/Sundial,DX4334-008,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,0 Stars
1,Nike Air VaporMax Plus,Men's Shoes,$210,https://static.nike.com/a/images/t_PDP_LOADING...,The Nike Air VaporMax Plus looks to the past a...,Black/Black/Dark Grey,924453-004,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.9 Stars
2,Nike Air Max 270,Men's Shoes,$160,https://static.nike.com/a/images/t_PDP_LOADING...,Nike's first lifestyle Air Max brings you styl...,White/White/Black,AH8050-100,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars
3,Air Jordan 5 Retro SE Craft,Men's Shoes,$210,https://static.nike.com/a/images/t_PDP_LOADING...,"Bring some pop to your look with the AJ5, the ...",Light Orewood Brown/Flat Pewter/Sail/Safety Or...,FD9222-180,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,5 Stars
4,Zoom Freak 4,Basketball Shoes,$130,https://static.nike.com/a/images/t_PDP_LOADING...,Giannis is an incessant storm of stamina and s...,Indigo Haze/Blue Tint/Baltic Blue/Space Purple,DJ6149-500,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars
5,Nike Air Max Pulse,Men's Shoes,$150,https://static.nike.com/a/images/t_PDP_LOADING...,The Air Max Pulse pulls inspiration from the L...,Cobblestone/Light Orewood Brown/Black/Reflect ...,DR0453-004,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.9 Stars
6,Air Jordan 6 Retro,Shoes,$200,https://static.nike.com/a/images/t_PDP_LOADING...,MJ's sixth signature shoe debuted during the 1...,Varsity Red/Black,CT8529-600,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.8 Stars
7,Nike Air Max 1 '86 OG G NRG,Men's Golf Shoes,$170,https://static.nike.com/a/images/t_PDP_LOADING...,The holy grail of fresh hits the links in the ...,White/Hot Punch/Sanddrift/Gridiron,DX8437-106,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,0 Stars
8,Nike Killshot 2 Leather,Men's Shoe,$90,https://static.nike.com/a/images/t_PDP_LOADING...,Inspired by the original low-profile tennis sh...,Sail/Gum Yellow/Midnight Navy,432997-107,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.7 Stars
9,Nike React Pegasus Trail 4 SE,Men's Trail Running Shoes,$140,https://static.nike.com/a/images/t_PDP_LOADING...,"Running is your daily ritual, taking you from ...",Pure Platinum/Baltic Blue/Black/Green Abyss,FB7182-001,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.6 Stars


## **ImportantInformationOfNikeWebOfMenShowesReviews**

In [ ]:
nike_men_shows_review_df = pd.concat(review_dataframe_names, ignore_index=True)
nike_men_shows_review_df

,product_reviwe_title,author,date,reviwe
0,Nike Air Vapor max plus,Krista339132879,"May 21, 2023",I love these shoes and this is my 2nd purchase...
1,He loves it,Ok1kidkog,"May 03, 2023",[This review was collected as part of a promot...
2,Great features,Cbeech mallen,"May 03, 2023","Product received for free, or reviewed as part..."
3,Awesome,Steve420,"May 11, 2023",[This review was collected as part of a promot...
4,Comfortable,Thebosslady,"May 10, 2023","Product received for free, or reviewed as part..."
5,Comfortable Fit,CSB95,"May 09, 2023","I love how these shoes look on my feet, they g..."
6,Fire 🔥,Trubz118,"May 23, 2023","Comfy and stylish, we need more colors for thi..."
7,,javierr906855513,"May 23, 2023",I think we sleepin on these. Soft materials ma...
8,Amazing design,Edgardo586187082,"May 24, 2023",They're stilysh. You could totally wear them a...
9,good shoe,lilcp6,"May 21, 2023","great shoe, real comfy and light even tho gian..."


In [ ]:
# Create a list to store the non-matching URLs
non_matching_urls = []

# Iterate over each URL in used_url_ids
for url in used_url_ids:
    # Check if the URL is not present in the 'shoes_catalog_link' column
    if url not in Nike_df_men_home['shoes_catalog_link'].tolist():
        non_matching_urls.append(url)

# Print the non-matching URLs
print(f"{len(non_matching_urls)}\n{non_matching_url}")
for this I want to see the serial number of the links which are same address

3
['https://www.nike.com/t/pegasus-40-mens-road-running-shoes-MCGW9h/FB7180-001', 'https://www.nike.com/t/air-max-97-mens-shoes-LJmK45/921826-101', 'https://www.nike.com/t/air-max-plus-mens-shoes-x9G2xF/604133-139']


In [ ]:
nike_men_shows_df[nike_men_shows_df.price=="$175"]

,product_name,category,price,image,description,colors,style,policy,order_link,rating
19,Nike Air Max 97,Men's Shoes,$175,https://static.nike.com/a/images/t_PDP_LOADING...,Featuring the original ripple design inspired ...,White/Black/Wolf Grey,921826-101,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.8 Stars
21,Nike Air Max Plus,Men's Shoes,$175,https://static.nike.com/a/images/t_PDP_LOADING...,Let your attitude have the edge in your Nike A...,White/Black/Cool Grey/White,604133-139,Free standard shipping on orders $50+ and free...,https://www.nike.com/help/a/free-shipping,4.2 Stars
